# EOPF Product Data Structure samples
This is a simple tutorial to show you how to build a Generic Product Data Structure and display it using our template <br>
Do not hesitate to modify/update the code and do new suggestions


In [12]:
import xarray as xr
import glob
from utils import display, get_ds_variables as get_ds

from EOProductDataStructure import EOProductBuilder, EOVariableBuilder, EOGroupBuilder

# Let's start building a new Product Data Structure
# 1. Create an instance of EOProduct

In [13]:

# Adding global attributes
global_attributes = [
    "product_name",
    "creation_time",
    "start_time",
    "stop_time",
    "footprint",
    "absolute_orbit_number",
    "ac_subsampling_factor",
    "al_subsampling_factor",
    "comment",
    "contact",
    "history",
    "institution",
    "references",
    "resolution",
    "source",
    "title"]

attrs = {att: 'TBD' for att in global_attributes}

# Create an instance of EOProduct , give it a name
p = EOProductBuilder(name="S3 OLCI L1 Product Data Structure", coords=EOGroupBuilder('coords'), attrs=attrs)

# display this empty product
display(p.compute())

# 2. Adding Groups to our Product

In [14]:
# adding groups to our product
radiances = EOGroupBuilder("radiances", coords=EOGroupBuilder('coordinates'))
tie_points = EOGroupBuilder("tie_points")
removed_pixels = EOGroupBuilder("removed_pixels")
instrument_data = EOGroupBuilder("instrument_data")

#p.groups.append(radiances)
#p.groups.append(tie_points)
p.groups += [radiances,tie_points,removed_pixels,instrument_data]

# again display the product
display(p.compute())

# 3. Adding Dimensions/Coordinates and Variables to Groups

In [15]:
# adding attributes to radiances group
radiances.attrs['description'] = 'Radiances Data Group'
# adding dimensions
radiances.dims = ['rows','columns']
# ======================= Coordinates =======================

for name in ["latitude","longitude","altitude","time_stamp"]:
    coord = EOVariableBuilder(name, default_attrs=True) # set default_attrs to False to disable default attributes
    coord.dtype = "int32"
    coord.dims = radiances.dims
    radiances.coords.variables.append(coord)

# ======================= Variables =======================
for r in range(21):
    variable = EOVariableBuilder("Oa%02d_radiance" % (r+1), default_attrs = True)
    variable.dtype = "uint16"
    variable.dims = radiances.dims
    radiances.variables.append(variable)
    
display(p.compute())

# 4. Adding Metadata 

In [16]:
p.metadatas = ['xdfumanifest.json','metadata.file']
display(p.compute())

# 5. Example of S3 product conversion 

In [17]:
# Path ot S3 product
path_to_product = glob.glob("data/S3A_OL_1*.SEN3")[0]
path_to_product

'data/S3A_OL_1_EFR____20200101T101517_20200101T101817_20200102T141102_0179_053_179_2520_LN1_O_NT_002.SEN3'

# 5.1 Define a new Data Structure

In [18]:
# Groups definition
groups = {}
groups['radiances'] = get_ds(path_to_product, names=["Oa%02d_radiance" %r for r in range(21)] + ["time_coordinates","geo_coordinates"], process_coords=True)
groups['tie_points'] = get_ds(path_to_product, names=["tie_geo_coordinates","tie_meteo","tie_geometries"], process_coords=True)
groups['removed_pixels'] = get_ds(path_to_product, names=["removed_pixels"], process_coords=True)
groups['instrument_data'] = get_ds(path_to_product, names=["instrument_data"], process_coords=True)
groups['quality_flags'] = get_ds(path_to_product, names=["qualityFlags"], process_coords=True)


# Create a new EOProduct instance
product = EOProductBuilder("S3 OLCI L1 Product Data Structure", coords=EOGroupBuilder('coords'))
# do the same work as before 
product.metadatas = ["xfdumanifest.json"]

# global attributes
#get them from a netcdf file 
ds = xr.open_dataset(path_to_product + "/Oa01_radiance.nc",decode_times=False,mask_and_scale=False)
product.attrs.update(ds.attrs)
# ==================== Product groups setting ========================

for group_name, ds in groups.items():
    ds = groups[group_name]
    group = EOGroupBuilder(group_name, coords=EOGroupBuilder('coordinates'))
    group.attrs["description"] = f"{group_name} Data Group"
    group.dims = ds.dims

    for cname, cood in ds.coords.items():

        variable = EOVariableBuilder(
            cname, dtype=cood.dtype, dims=cood.dims, attrs=cood.attrs, default_attrs = False,
        )
        group.coords.variables.append(variable)


    for v, var in ds.variables.items():
        if v not in ["latitude","longitude","altitude","time_stamp"]:
            variable = EOVariableBuilder(
                v, dtype=var.dtype, dims=var.dims, attrs=var.attrs, default_attrs = False
            )
            group.variables.append(variable)

    product.groups.append(group)

In [19]:
display(product.compute())


# New Proposal : Put all the coordinates in the same group => Coordinates

In [20]:

p = product

rads = p.groups[0]
tps = p.groups[1]

#Create a dedicated coordiante group , call it Coordinates
coordinates = EOGroupBuilder("Coordinates")

coordinates.variables = rads.coords.variables

for c in rads.coords.variables:
    for d in c.dims:
        if d not in coordinates.dims:
            coordinates.dims.append(d)
            
for c in tps.coords.variables:
    c.name = f'tie_{c.name}'
    for d in c.dims:
        if d not in coordinates.dims:
            coordinates.dims.append(d)
            
coordinates.variables += tps.coords.variables

rads.coords = None
tps.coords = None
    
p.coords = coordinates
p.attrs['metadata_files'] = '[xfdumanfist.json]'


In [21]:
display(p.compute())